
# Left and Right Inverses of a Matrix 



## Quick definitions
- A **left inverse** of $A$ is a matrix $L$  such that $LA = I$. It exists when $A$ has **full column rank** (e.g., a *tall* matrix, more rows than columns). $m$>>$n$
- A **right inverse** of $A$ is a matrix $R$ such that $AR = I$. It exists when $A$ has **full row rank** (e.g., a *wide* matrix, more columns than rows). $m$<<$n$
- When $A$ is square and invertible, $A^{-1}$ is both a left and a right inverse.

**Least squares (tall matrices):** For full column rank, the canonical left inverse is $L = (A^\top A)^{-1}A^\top$. Then the least‑squares solution to $Ax = b$ is $x = Lb$, and $Ax$ is the orthogonal **projection of $b$ onto Col(A)**.

**Minimum norm (wide matrices):** For full row rank, a right inverse is $R = A^\top(AA^\top)^{-1}$. Then one solution of $Ax = b$ is $x = Rb$. 


 


When to prefer computing the right inverse directly
1) The matrix is wide $(m≪n)$ and full row rank
If A is $m\times n$ with $m≪n$, computing the right inverse via
$R=A^⊤(AA^⊤)^{−1}$
requires inverting only an $m\times m$ matrix $AA^⊤$, which is much cheaper than anything that touches an $n \times n$ object.




Example scenario:
$m=100$, $n=100,000$.

$AA^⊤$ is $100\times 100$ →  tractable.
$A^⊤A$ is $100,000\times 100,000$ → infeasible.


## Setup
We will use only NumPy to keep things lightweight. Utility helpers are included to check identities and orthogonality numerically (within a tolerance).


In [11]:

import numpy as np

def is_close(A, B, tol=1e-9):
    return np.allclose(A, B, atol=tol, rtol=0)

def print_matrix(name, M):
    print(f"{name}\nShape={M.shape}\n{M}")

def orthogonality_check(A, r, tol=1e-9):
    v = A.T @ r
    print(f"||A^T r||_2 = {np.linalg.norm(v):.3e} (should be ~ 0 if residual is orthogonal to col(A)) ")

np.set_printoptions(precision=4, suppress=True)



## 1) Tall matrix (full column rank) → Left inverse & least squares geometry
**Theory:** If $A\in\mathbb{R}^{m\times n}$ with $m>n$ and full column rank, then $L=(A^\top A)^{-1}A^\top$ satisfies $LA=I_n$. For any $b\in\mathbb{R}^m$, the least‑squares solution is $x=Lb$. The fitted vector $\hat b=Ax$ is the **orthogonal projection** of $b$ onto $\\operatorname{col}(A)$, and the residual $r=b-\hat b$ is orthogonal to that space (i.e., $A^\top r=0$).


In [5]:
# A simple tall matrix (3x2) with full column rank
A_tall = np.array([[1., 2.],
                   [3., 4.],
                   [5., 7.]])

# Left inverse via normal equations (for didactic purposes)
L = np.linalg.inv(A_tall.T @ A_tall) @ A_tall.T

print_matrix('A_tall', A_tall)
print_matrix('Left inverse L = (A^T A)^{-1} A^T', L)
print('Check L A = I: ', is_close(L @ A_tall, np.eye(A_tall.shape[1])))

# Least squares for Ax = b
b = np.array([2., 1., -1.])
x_ls = L @ b
b_hat = A_tall @ x_ls
r = b - b_hat

print_matrix('b', b)
print_matrix('x_ls (least squares)', x_ls)
print_matrix('b_hat = A x_ls (projection of b onto col(A))', b_hat)
print_matrix('Residual r = b - b_hat', r)
orthogonality_check(A_tall, r)


A_tall shape=(3, 2) [[1. 2.]
 [3. 4.]
 [5. 7.]] 
Left inverse L = (A^T A)^{-1} A^T shape=(2, 3) [[-2.0714  0.7857  0.1429]
 [ 1.5    -0.5     0.    ]] 
Check L A = I:  True
b shape=(3,) [ 2.  1. -1.] 
x_ls (least squares) shape=(2,) [-3.5  2.5] 
b_hat = A x_ls (projection of b onto col(A)) shape=(3,) [ 1.5 -0.5 -0. ] 
Residual r = b - b_hat shape=(3,) [ 0.5  1.5 -1. ] 
||A^T r||_2 = 8.967e-14 (should be ~ 0 if residual is orthogonal to col(A)) 



## 2) Wide matrix (full row rank) → Right inverse & families of solutions
**Theory:** If $A\in\mathbb{R}^{m\times n}$ with $m<n$ and full row rank, then $R=A^\top(AA^\top)^{-1}$ satisfies $AR=I_m$. Given $b\in\mathbb{R}^m$, one solution to $Ax=b$ is $x=Rb$. In fact, **all** solutions are $x=Rb+z$ for any $z\in\\mathcal{N}(A)$ (the nullspace). The pseudoinverse $A^+$ returns the **minimum‑norm** solution among these.


In [26]:

# A simple wide matrix (2x3) with full row rank
A_wide = np.array([[1., 2., 3.],
                  [4., 5., 6.]])

# Right inverse via normal equations on rows (for didactic purposes)
R = A_wide.T @ np.linalg.inv(A_wide @ A_wide.T)

print_matrix('A_wide', A_wide)
print_matrix('Right inverse R = A^T (A A^T)^{-1}', R)
print('Check A R = I: ', is_close(A_wide @ R, np.eye(A_wide.shape[0])))

# Solve Ax = b
b2 = np.array([1., -1.])
x0 = R @ b2
print_matrix('b2', b2)
print_matrix('One solution x0 = R b2', x0)
print_matrix('A x0 (should equal b2)', A_wide @ x0)

# Nullspace via SVD: right singular vectors corresponding to ~zero singular values
U, s, Vt = np.linalg.svd(A_wide, full_matrices=True)
# For a 2x3 full row rank matrix, nullity = 1; the null vector is the last row of Vt (since s sorted desc)
nvec = Vt[-1, :].reshape(-1, 1)  # (3,1)
# Verify it's in the nullspace
print_matrix('A_wide @ nvec (should be ~0)', A_wide @ nvec)
# Family of solutions: x = x0 + nvec * t
t = 3.0
x_family = x0.reshape(-1,1) + nvec * t
print_matrix('x_family (t=3)', x_family)
print_matrix('A x_family (should equal b2)', A_wide @ x_family)


A_wide
Shape=(2, 3)
[[1. 2. 3.]
 [4. 5. 6.]]

Right inverse R = A^T (A A^T)^{-1}
Shape=(3, 2)
[[-0.9444  0.4444]
 [-0.1111  0.1111]
 [ 0.7222 -0.2222]]

Check A R = I:  True
b2
Shape=(2,)
[ 1. -1.]

One solution x0 = R b2
Shape=(3,)
[-1.3889 -0.2222  0.9444]

A x0 (should equal b2)
Shape=(2,)
[ 1. -1.]

A_wide @ nvec (should be ~0)
Shape=(2, 1)
[[ 0.]
 [-0.]]

x_family (t=3)
Shape=(3, 1)
[[-0.1641]
 [-2.6717]
 [ 2.1692]]

A x_family (should equal b2)
Shape=(2, 1)
[[ 1.]
 [-1.]]




## 3) The transpose trick
**Key idea:** If $A$ is tall full‑column‑rank and you computed the left inverse $L=(A^\top A)^{-1}A^\top$, then for **the transposed problem** $A^\top$ (which is wide full‑row‑rank), the matrix **$L^\top$ is a right inverse of $A^\top$**, because

$$
(A^\top)\,L^\top = (LA)^\top = I^\top = I.
$$

This is handy if an algorithm needs both forms but building only one is cheaper or already available.


In [7]:

# Verify the transpose trick with our tall example
A = A_tall
L = np.linalg.inv(A.T @ A) @ A.T
check = A.T @ L.T
print_matrix('(A^T) * (L^T)', check)
print('Is it identity?', is_close(check, np.eye(A.shape[1])))


(A^T) * (L^T) shape=(2, 2) [[1. 0.]
 [0. 1.]] 
Is it identity? True


Exercises:
1. For a large tall matrix, find right inverse using left inverse (Hint: Use transpose idea). Verify your answer. 
Give an explicit illustration with $3 \times 2$ matrix.
2. For a large wide matrix, find left inverse using right invese. Verify your answer. Give an explicit illustration with $2 \times 3$ matrix.

In [35]:
A = np.array([[1.,0.],
              [0.,1.],
              [1.,1.]])

# Left inverse of A
L = np.linalg.inv(A.T @ A) @ A.T

# Right inverse of A^T via transpose trick
R = L.T                # same as A @ np.linalg.inv(A.T @ A)
R_alt = A @ np.linalg.inv(A.T @ A)

print("L = (A^T A)^{-1} A^T\n", L)
print("\nR = L^T (right inverse of A^T)\n", R)
print("\nR_alt (A (A^T A)^{-1})\n", R_alt)

print("\nCheck A^T @ R (should be I_2):\n", np.round(A.T @ R, 10))


L = (A^T A)^{-1} A^T
 [[ 0.6667 -0.3333  0.3333]
 [-0.3333  0.6667  0.3333]]

R = L^T (right inverse of A^T)
 [[ 0.6667 -0.3333]
 [-0.3333  0.6667]
 [ 0.3333  0.3333]]

R_alt (A (A^T A)^{-1})
 [[ 0.6667 -0.3333]
 [-0.3333  0.6667]
 [ 0.3333  0.3333]]

Check A^T @ R (should be I_2):
 [[1. 0.]
 [0. 1.]]


In [40]:
import numpy as np

# Large wide matrix (more columns than rows), full row rank
m, n = 5, 800  # 5 rows, 8 columns
A_wide = np.random.randn(m, n)

# Right inverse: A^T (A A^T)^(-1)
right_inverse = A_wide.T @ np.linalg.inv(A_wide @ A_wide.T)

# Now, use the transpose trick:
# Left inverse of A_wide is the transpose of right inverse of A_wide^T
A_T = A_wide.T
left_inverse_from_transpose = (A_T.T @ np.linalg.inv(A_T @ A_T.T)).T

# Verification: LHS * A = I (m x m)
check = left_inverse_from_transpose @ A_wide
print("Left inverse (from transpose idea) * A =\n", np.round(check, 3))


Left inverse (from transpose idea) * A =
 [[-21.937  -7.079  -4.613 ... -41.214  36.673  41.391]
 [  2.15    6.682   1.73  ...   9.293 -12.467  -9.427]
 [-42.692  36.399  17.871 ... -16.389  -4.075  31.722]
 ...
 [ 23.112 -46.1    18.874 ...  25.64   33.51    5.58 ]
 [ -0.79    6.444 -10.617 ... -12.528  -0.533   4.615]
 [  2.663   0.287  -1.645 ...   1.486  -3.236  -2.6  ]]
